# Extração e Análise dos Dados - IPEA - Remuneração líquida média mensal no Executivo civil federal ativo, por sexo e raça - 1999-2020)

A metodologia aplicada para a atividade foi a de codificação e disponibilização dos dados em sequência lógica, para posteriormente comentá-los, ao fim do notebook.

Neste notebook, foram criados alguns dataframes de modo a facilitar a inferência dos dados de interesse, de acordo com a estratégia mais conveniente para extração de cada informação.

Pequenos ajustes podem ser feitos no código para modificar a apresentação dos dados, e ainda exportá-lo para novos arquivos. Fez-se uma ponderação entre a extração do maior número de dados possíveis e a simplicidade do código, de modo a não deixá-lo complexo demais, evitando a utilização de demais bibliotecas.

Obs: os termos utilizados na atividade como "Raça" e "Sexo" (sic), foram herdados da construção dos dados disponibilizados.

## Importação de Bibliotecas Necessárias

In [244]:
import pandas as pd
import requests
import os

## Configurando o ambiente do Projeto e o Acesso aos Dados

In [144]:
#Criação do diretório do projeto, com checagem de existência prévia

os.makedirs('./projeto-selecao-gc', exist_ok=True)
path = './projeto-selecao-gc'
os.chdir(path)

In [145]:
# Definição do caminho online

url = "https://www.ipea.gov.br/atlasestado/arquivos/downloads/5233-liquidosexoraca.csv"

# Download dos dados para repositório para documentação

response = requests.get(url)
open("dados.csv", "wb").write(response.content)

3003

## Transformação e Limpeza dos Dataframes

In [146]:
#Construção do Dataframe inicial

df_remuneracao = pd.read_csv("./dados.csv", sep = ";", decimal = ",")

In [147]:
#Visualizar informações gerais do dataframe montado

df_remuneracao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88 entries, 0 to 87
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  88 non-null     int64  
 1   ano         88 non-null     int64  
 2   sexo_raça   88 non-null     object 
 3   liquido     88 non-null     float64
dtypes: float64(1), int64(2), object(1)
memory usage: 2.9+ KB


In [148]:
#Visualizar cabeçalho do Dataframe

df_remuneracao.head()

,Unnamed: 0,ano,sexo_raça,liquido
0,1,1999,Homem Branco,6527.2
1,2,1999,Homem Negro,4768.7
2,3,1999,Mulher Branca,5261.6
3,4,1999,Mulher Negra,4010.0
4,5,2000,Homem Branco,5690.3


In [149]:
# Renomear as colunas para uma apresentação mais limpa dos dados
## RLMM: Remuneração Líquida Média Mensal

df_remuneracao = df_remuneracao.rename(columns={    
 'Unnamed: 0': 'Índice',
 'ano': 'Ano',
 'sexo_raça': 'Sexo Raça',
 'liquido': 'RLMM'}
)

In [150]:
# Visualização do topo do dataframe

df_remuneracao.head()

,Índice,Ano,Sexo Raça,RLMM
0,1,1999,Homem Branco,6527.2
1,2,1999,Homem Negro,4768.7
2,3,1999,Mulher Branca,5261.6
3,4,1999,Mulher Negra,4010.0
4,5,2000,Homem Branco,5690.3


In [151]:
#Separação das Colunas Sexo e Raça para análise isolada posteriormente -> Remodelagem linha coluna para Raça e Sexo

df_remuneracao[['Sexo', 'Raça']] = df_remuneracao['Sexo Raça'].str.split(' ', 1, expand=True)
df_remuneracao['Raça'] = df_remuneracao['Raça'].replace(['Branco', 'Negro'], ['Branca', 'Negra'])
df_remuneracao = df_remuneracao[['Ano', 'Sexo', 'Raça', 'RLMM']]

In [163]:
# Visualização do Topo do Dataframe Criado
df_remuneracao.head()

,Ano,Sexo,Raça,RLMM
0,1999,Homem,Branca,6527.2
1,1999,Homem,Negra,4768.7
2,1999,Mulher,Branca,5261.6
3,1999,Mulher,Negra,4010.0
4,2000,Homem,Branca,5690.3


In [222]:
# Visualização geral dos dados de forma transposta

df_remuneracao.groupby(['Ano', 'Sexo', 'Raça'])['RLMM'].agg(['mean']).transpose()

# Caso se deseje alterar a visualização dos dados:
#
# df_remuneracao.groupby(['Ano', 'Sexo', 'Raça'])['RLMM'].agg(['mean'])
#
# Caso se deseje visualizar todo o dataframe, configurar utilizando:
#
#with pd.option_context('display.max_rows', None,
                       #'display.max_columns', None,
                       #'display.precision', 3) 

Ano     1999                            2000                            2001  \
Sexo   Homem          Mulher           Homem          Mulher           Homem   
Raça  Branca   Negra  Branca   Negra  Branca   Negra  Branca   Negra  Branca   
mean  6527.2  4768.7  5261.6  4010.0  5690.3  4151.7  4887.3  3782.5  5833.8   

Ano           ...    2018            2019                            2020  \
Sexo          ...  Mulher           Homem          Mulher           Homem   
Raça   Negra  ...  Branca   Negra  Branca   Negra  Branca   Negra  Branca   
mean  4107.2  ...  7952.3  6048.9  9044.7  6454.7  7977.5  5996.2  8774.2   

Ano                           
Sexo          Mulher          
Raça   Negra  Branca   Negra  
mean  6272.2  7753.8  5815.5  

[1 rows x 88 columns]

In [181]:
# Distinguir dataframes por sexo e raça

df_rem_genero = df_remuneracao[['Ano','Sexo', 'RLMM']]
df_rem_raca = df_remuneracao[['Ano','Raça', 'RLMM']]

## Disponibilização de Dados para Análise Exploratória

In [318]:
# Obter informações Médias por Sexo Ano a Ano

df_rem_genero.groupby(['Ano', 'Sexo']).mean()

RLMM
Ano  Sexo           
1999 Homem   5647.95
     Mulher  4635.80
2000 Homem   4921.00
     Mulher  4334.90
2001 Homem   4970.50
     Mulher  4366.00
2002 Homem   5448.30
     Mulher  4749.70
2003 Homem   5215.35
     Mulher  4514.30
2004 Homem   5137.55
     Mulher  4412.85
2005 Homem   5436.20
     Mulher  4667.30
2006 Homem   5298.10
     Mulher  4614.25
2007 Homem   5932.00
     Mulher  5066.90
2008 Homem   6067.35
     Mulher  5149.20
2009 Homem   7000.60
     Mulher  6088.70
2010 Homem   7716.15
     Mulher  6990.35
2011 Homem   7765.55
     Mulher  7118.65
2012 Homem   7434.15
     Mulher  6874.50
2013 Homem   7514.20
     Mulher  7005.20
2014 Homem   7905.90
     Mulher  7328.15
2015 Homem   7650.55
     Mulher  7000.90
2016 Homem   7308.60
     Mulher  6761.30
2017 Homem   7889.10
     Mulher  7192.00
2018 Homem   7648.50
     Mulher  7000.60
2019 Homem   7749.70
     Mulher  6986.85
2020 Homem   7523.20
     Mulher  6784.65

In [236]:
# Resumo por raça Ano a Ano

df_rem_raca.groupby(['Raça', 'Ano'])['RLMM'].agg(['max','min','mean','median'])

max     min     mean   median
Raça   Ano                                   
Branca 1999  6527.2  5261.6  5894.40  5894.40
       2000  5690.3  4887.3  5288.80  5288.80
       2001  5833.8  4997.5  5415.65  5415.65
       2002  6407.0  5429.7  5918.35  5918.35
       2003  6133.3  5171.9  5652.60  5652.60
       2004  5902.9  4963.7  5433.30  5433.30
       2005  6314.4  5298.2  5806.30  5806.30
       2006  6227.3  5269.9  5748.60  5748.60
       2007  7049.0  5822.3  6435.65  6435.65
       2008  7220.2  5946.1  6583.15  6583.15
       2009  8347.7  7049.5  7698.60  7698.60
       2010  9115.6  8005.6  8560.60  8560.60
       2011  9200.9  8153.7  8677.30  8677.30
       2012  8757.8  7843.2  8300.50  8300.50
       2013  8832.8  7964.9  8398.85  8398.85
       2014  9279.1  8321.2  8800.15  8800.15
       2015  8996.8  7953.9  8475.35  8475.35
       2016  8506.6  7647.4  8077.00  8077.00
       2017  9186.3  8144.7  8665.50  8665.50
       2018  8893.2  7952.3  8422.75  8422.75
       2019  9044.7  7977.5  8511.10  8511.10
       2020  8774.2  7753.8  8264.00  8264.00
Negra  1999  4768.7  4010.0  4389.35  4389.35
       2000  4151.7  3782.5  3967.10  3967.10
       2001  4107.2  3734.5  3920.85  3920.85
       2002  4489.6  4069.7  4279.65  4279.65
       2003  4297.4  3856.7  4077.05  4077.05
       2004  4372.2  3862.0  4117.10  4117.10
       2005  4558.0  4036.4  4297.20  4297.20
       2006  4368.9  3958.6  4163.75  4163.75
       2007  4815.0  4311.5  4563.25  4563.25
       2008  4914.5  4352.3  4633.40  4633.40
       2009  5653.5  5127.9  5390.70  5390.70
       2010  6316.7  5975.1  6145.90  6145.90
       2011  6330.2  6083.6  6206.90  6206.90
       2012  6110.5  5905.8  6008.15  6008.15
       2013  6195.6  6045.5  6120.55  6120.55
       2014  6532.7  6335.1  6433.90  6433.90
       2015  6304.3  6047.9  6176.10  6176.10
       2016  6110.6  5875.2  5992.90  5992.90
       2017  6591.9  6239.3  6415.60  6415.60
       2018  6403.8  6048.9  6226.35  6226.35
       2019  6454.7  5996.2  6225.45  6225.45
       2020  6272.2  5815.5  6043.85  6043.85

In [241]:
# Resumo por raça Ano a Ano

df_rem_genero.groupby(['Sexo', 'Ano'])['RLMM'].agg(['max','min','mean','median'])

max     min     mean   median
Sexo   Ano                                   
Homem  1999  6527.2  4768.7  5647.95  5647.95
       2000  5690.3  4151.7  4921.00  4921.00
       2001  5833.8  4107.2  4970.50  4970.50
       2002  6407.0  4489.6  5448.30  5448.30
       2003  6133.3  4297.4  5215.35  5215.35
       2004  5902.9  4372.2  5137.55  5137.55
       2005  6314.4  4558.0  5436.20  5436.20
       2006  6227.3  4368.9  5298.10  5298.10
       2007  7049.0  4815.0  5932.00  5932.00
       2008  7220.2  4914.5  6067.35  6067.35
       2009  8347.7  5653.5  7000.60  7000.60
       2010  9115.6  6316.7  7716.15  7716.15
       2011  9200.9  6330.2  7765.55  7765.55
       2012  8757.8  6110.5  7434.15  7434.15
       2013  8832.8  6195.6  7514.20  7514.20
       2014  9279.1  6532.7  7905.90  7905.90
       2015  8996.8  6304.3  7650.55  7650.55
       2016  8506.6  6110.6  7308.60  7308.60
       2017  9186.3  6591.9  7889.10  7889.10
       2018  8893.2  6403.8  7648.50  7648.50
       2019  9044.7  6454.7  7749.70  7749.70
       2020  8774.2  6272.2  7523.20  7523.20
Mulher 1999  5261.6  4010.0  4635.80  4635.80
       2000  4887.3  3782.5  4334.90  4334.90
       2001  4997.5  3734.5  4366.00  4366.00
       2002  5429.7  4069.7  4749.70  4749.70
       2003  5171.9  3856.7  4514.30  4514.30
       2004  4963.7  3862.0  4412.85  4412.85
       2005  5298.2  4036.4  4667.30  4667.30
       2006  5269.9  3958.6  4614.25  4614.25
       2007  5822.3  4311.5  5066.90  5066.90
       2008  5946.1  4352.3  5149.20  5149.20
       2009  7049.5  5127.9  6088.70  6088.70
       2010  8005.6  5975.1  6990.35  6990.35
       2011  8153.7  6083.6  7118.65  7118.65
       2012  7843.2  5905.8  6874.50  6874.50
       2013  7964.9  6045.5  7005.20  7005.20
       2014  8321.2  6335.1  7328.15  7328.15
       2015  7953.9  6047.9  7000.90  7000.90
       2016  7647.4  5875.2  6761.30  6761.30
       2017  8144.7  6239.3  7192.00  7192.00
       2018  7952.3  6048.9  7000.60  7000.60
       2019  7977.5  5996.2  6986.85  6986.85
       2020  7753.8  5815.5  6784.65  6784.65

In [321]:
# Razão de Aumento percentual Ano a Ano - com base no Sexo

df_rem_genero.groupby(['Sexo', 'Ano']).mean().pct_change()

RLMM
Sexo   Ano           
Homem  1999       NaN
       2000 -0.128710
       2001  0.010059
       2002  0.096127
       2003 -0.042756
       2004 -0.014918
       2005  0.058131
       2006 -0.025404
       2007  0.119647
       2008  0.022817
       2009  0.153815
       2010  0.102213
       2011  0.006402
       2012 -0.042676
       2013  0.010768
       2014  0.052128
       2015 -0.032299
       2016 -0.044696
       2017  0.079427
       2018 -0.030498
       2019  0.013231
       2020 -0.029227
Mulher 1999 -0.383799
       2000 -0.064908
       2001  0.007174
       2002  0.087884
       2003 -0.049561
       2004 -0.022473
       2005  0.057661
       2006 -0.011366
       2007  0.098098
       2008  0.016243
       2009  0.182456
       2010  0.148086
       2011  0.018354
       2012 -0.034297
       2013  0.019012
       2014  0.046101
       2015 -0.044657
       2016 -0.034224
       2017  0.063701
       2018 -0.026613
       2019 -0.001964
       2020 -0.028940

In [228]:
# Razão de Aumento percentual Ano a Ano - com base na Raça

df_rem_raca.groupby(['Raça','Ano']).agg(['mean']).pct_change()

RLMM
                 mean
Raça   Ano           
Branca 1999       NaN
       2000 -0.102742
       2001  0.023985
       2002  0.092824
       2003 -0.044903
       2004 -0.038796
       2005  0.068651
       2006 -0.009937
       2007  0.119516
       2008  0.022919
       2009  0.169440
       2010  0.111968
       2011  0.013632
       2012 -0.043424
       2013  0.011849
       2014  0.047780
       2015 -0.036908
       2016 -0.047001
       2017  0.072861
       2018 -0.028013
       2019  0.010489
       2020 -0.029033
Negra  1999 -0.468859
       2000 -0.096199
       2001 -0.011658
       2002  0.091511
       2003 -0.047340
       2004  0.009823
       2005  0.043744
       2006 -0.031055
       2007  0.095947
       2008  0.015373
       2009  0.163444
       2010  0.140093
       2011  0.009925
       2012 -0.032021
       2013  0.018708
       2014  0.051196
       2015 -0.040069
       2016 -0.029663
       2017  0.070533
       2018 -0.029498
       2019 -0.000145
       2020 -0.029171

In [ ]:
# Razão de Aumento percentual Ano a Ano - com base na Raça e no Sexo

df_remuneracao.groupby(['Raça', 'Sexo', 'Ano'])[['RLMM']].mean().pct_change()

In [223]:
# Resumo por Raça

df_rem_raca.groupby(['Raça'])['RLMM'].agg(['max','min','mean','median'])

,max,min,mean,median
Raça,,,,
Branca,9279.1,4887.3,7228.568182,7700.6
Negra,6591.9,3734.5,5263.411364,5734.5


In [270]:
# Criacao de modelo de Raça com Variaveis Dummy

df_dummy = df_remuneracao
df_dummy = df_dummy.replace(['Branca', 'Negra'], ['1', '0'])
df_dummy = df_dummy.replace(['Homem', 'Mulher'], ['1', '0'])
df_dummy = df_dummy.astype({"Raça": int,
               "Sexo": int}, errors='raise')

In [272]:
# Calculo de Correlação entre as variáveis

df_raca_dummy.corr()

,Ano,Sexo,Raça,RLMM
Ano,1.000000e+00,1.119972e-15,1.129517e-15,0.651910
Sexo,1.119972e-15,1.000000e+00,-2.523234e-17,0.224562
Raça,1.129517e-15,-2.523234e-17,1.000000e+00,0.624850
RLMM,6.519095e-01,2.245617e-01,6.248500e-01,1.000000


# Análise dos Dados

A partir da extração dos dados e da sua posterior análise, podemos identificar algumas tendências na série temporal que relaciona o salário líquido mensal médio com atributos como Raça e Sexo.

## Pôde-se inferir a partir dos dados que entre os anos de 1999 e 2020:

Ocorreu um crescimento na RLMM em média de:

- 33,20% para Homens em geral,
- 46,35% para Mulheres em geral

Estratificando por raça e sexo, temos os seguintes índices de crescimento:
- Homens brancos: 34,43%
- Homens negros: 31,53%
- Mulheres brancas: 47,37%
- Mulheres negras: 45,02

A partir disso, podemos perceber que as mulheres tiveram um crescimento salarial maior que o dos homens no período analisado, sendo que as mulheres brancas tiveram em média um crescimento maior que as negras. Analisando os dados isoladamente, entretanto, pode-se perceber que apesar de ter uma tendência equiparação salarial aos homens, as mulheres ainda recebem em média menos que os homens, para ambas as "raças".
Para ambas as categorias de sexo, no que diz respeito à raça, o crescimento médio salarial ao longo dos anos ainda foi menor para indivíduos da raça "negra" em comparação com "brancos".
A partir da matriz de correlação, podemos analisar que os índices de correlação entre a RLMM e a Raça e o Sexo, e identificar que a correlação entre as variáveis é maior para a raça do que para o sexo.